In [7]:
import math
import numbers
import reprlib
from array import array


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        if isinstance(other, Vector):
            return len(self) == len(other) and all(a == b for a, b in zip(self, other))
        else:
            raise NotImplemented

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, int):
            return self._components[index]
        else:
            raise TypeError(f'{cls.__name__} indices must be integers.')

    def __getattr__(self, name):
        return self._components[int(name[1:])]

    def __setattr__(self, key, value):
        if key.startswith('x'):
            self._components[int(key[1:])] = value
        else:
            super().__setattr__(key, value)

    def __hash__(self):
        import functools, operator
        return functools.reduce(operator.xor, (hash(x) for x in self._components), 0)

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    def __add__(self, other):
        try:
            return Vector(a + b for a, b in zip(self, other))
        except TypeError:
            raise NotImplemented

    def __radd__(self, other):
        return self.__add__(other)

    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            raise NotImplemented

    def __rmul__(self, other):
        return self * other

    def __matmul__(self, other):
        try:
            return sum(a * b for a, b in zip(self, other))
        except TypeError:
            raise NotImplemented

    def __rmatmul__(self, other):
        return self @ other

In [8]:
a = Vector([1, 2])
b = Vector([3, 4])
c = Vector([5])
b @ a

11.0